In [38]:
import pyspark as ps
from  pyspark.sql.types import StructType,StructField,StringType
spark = (ps.sql.SparkSession
         .builder
         .master('local[4]')
         .appName('lecture')
         .getOrCreate()
        )
sc = spark.sparkContext

In [39]:
from pyspark.sql import SQLContext

In [40]:
sqlcontext=SQLContext(sc)
# df=sqlcontext.read.json('yelp_dataset/user.json')

In [41]:
business=sqlcontext.read.json('yelp_dataset/business.json')

In [42]:
review=sqlcontext.read.json('yelp_dataset/review.json')

In [43]:
review.printSchema()
#review["text"].show()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



In [44]:
type(review)

pyspark.sql.dataframe.DataFrame

In [ ]:
# schema = StructType([StructField('json', StringType(), True)])
# rdd = (review
#   .select('json')
#   .rdd
#   .flatMap(lambda x: x)
#   .flatMap(lambda x: json.loads(x))
#   .map(lambda x: x.get('body'))
# )
# new_df = sql_context.createDataFrame(rdd, schema)
# new_df.show()

In [45]:
review.createOrReplaceTempView("yelp")
business.createOrReplaceTempView("yelp_business")
#unique_business = spark.sql("select distinct business_id from yelp")
#unique_business.count()


In [46]:
unique_comment = spark.sql("select distinct text from yelp")
unique_comment.count()
yelp_review = spark.sql("SELECT business_id,useful,text FROM yelp WHERE useful>=1")

In [ ]:
yelp_review.show()

In [ ]:
yelp_review.count()

In [ ]:
yelp_review.printSchema()

In [ ]:
yelp_review.describe().show()

In [ ]:
from pyspark.sql import functions as F
df_out = yelp_review.groupBy("business_id").agg(F.sum("useful")).orderBy("sum(useful)", ascending=False)
df_out.show()


In [ ]:
df_out2 = yelp_review.groupBy("text").agg(F.sum("business_id"))
df_out2.show()

In [ ]:
df_out2 = yelp_review.select("business_id").distinct().count()
df_out2

In [ ]:
total_buisness=yelp_review.select("business_id").count()
total_buisness

In [ ]:
total_buisness/df_out2

In [ ]:
business.printSchema()

In [ ]:
business_view=business.select("business_id","name","categories","review_count","stars").show()#groupby("categories").count("review_count").orderBy("review_count").show()
business_view

In [ ]:
#yelp_review_useful0 = spark.sql("SELECT business_id,useful,text FROM yelp WHERE useful=0")
#yelp_review_useful0_count = spark.sql("SELECT b.business_id,b.name,count(y.useful) FROM yelp as y inner join business_view b group by y.business_id having count(y.useful)=0")

#yelp_review_useful0_count.show()

In [ ]:
#type(yelp_business)

In [ ]:
yelp_review_useful0.show()

In [ ]:
buss_category = spark.sql("SELECT distinct business_id,categories from yelp_business where categories like '%India%'")

In [ ]:
#buss_category.show(100)

In [ ]:
#buss_category.count()

In [ ]:
type(buss_category)

In [ ]:
#spark.sql(select buss_category)

In [ ]:
#business_view=business.select("business_id","name","categories","review_count","stars").show()#groupby("categories").count("review_count").orderBy("review_count").show()
review_name_category = spark.sql("select categories,name,count(name) from yelp_business group by categories")


In [49]:
buss_category2 = spark.sql("SELECT distinct b.business_id,b.categories,r.useful,r.text from yelp_business b inner join yelp r on r.business_id = b.business_id where categories like '%India%'")

In [50]:
#buss_category2.show()

In [51]:
#buss_category2.count()

In [52]:
#buss_cat_useful = buss_category2.select("useful")
buss_cat_useful=buss_category2.filter("useful >= 1")

In [53]:
buss_cat_useful.count()

38041

In [54]:
#buss_cat_useful = buss_category2.select("useful" ==0)
buss_not_cat_useful=buss_category2.filter("useful == 0")

In [55]:
buss_not_cat_useful.count()

41726

In [ ]:
buss_category2.write.format("csv").save("indian_review")


In [57]:
indian_rest_review_prediction=buss_category2.toPandas()

In [60]:
indian_rest_review_prediction.head(5)

,business_id,categories,useful,text
0,9OHxN88qy_BSmY_8yqWQ5w,"Indian, Halal, Vegetarian, Restaurants",1,We came here with a large group of family on a...
1,SU56w479vUfFHsvmvQIf7A,"Restaurants, Indian, Pakistani",1,The food was great. The flavors seemed authent...
2,xA6L2w_TlbxHubb1xeBE8w,"Food, Pakistani, Indian, Restaurants, Bakeries",1,I go back and forth on how I feel about Tamari...
3,xA6L2w_TlbxHubb1xeBE8w,"Food, Pakistani, Indian, Restaurants, Bakeries",1,Kodi vepudu... Is my favourite... will never m...
4,EfAqol3tWckyNrBMIooJmg,"Arabian, Indian, Restaurants, Pakistani, Halal",1,This place is really good. My wife and I were ...


In [61]:
def useful_not_useful(x):
    if x == 0:
        return 0
    else:
        return 1
    

In [62]:
indian_rest_review_prediction["label"]=indian_rest_review_prediction["useful"].apply(lambda x:useful_not_useful(x))

In [65]:
indian_rest_review_prediction.head(15)

,business_id,categories,useful,text,label
0,9OHxN88qy_BSmY_8yqWQ5w,"Indian, Halal, Vegetarian, Restaurants",1,We came here with a large group of family on a...,1
1,SU56w479vUfFHsvmvQIf7A,"Restaurants, Indian, Pakistani",1,The food was great. The flavors seemed authent...,1
2,xA6L2w_TlbxHubb1xeBE8w,"Food, Pakistani, Indian, Restaurants, Bakeries",1,I go back and forth on how I feel about Tamari...,1
3,xA6L2w_TlbxHubb1xeBE8w,"Food, Pakistani, Indian, Restaurants, Bakeries",1,Kodi vepudu... Is my favourite... will never m...,1
4,EfAqol3tWckyNrBMIooJmg,"Arabian, Indian, Restaurants, Pakistani, Halal",1,This place is really good. My wife and I were ...,1
5,-iFvYhgysvjkxckCr42NRw,"Restaurants, Food, Ethnic Food, Indian, Specia...",1,Totally amazed that this place offers free cha...,1
6,ohz8ljVxcZMo2bMGrBZfgw,"Vegetarian, Nightlife, Indian, Salad, Pakistan...",1,"The best Indian in Vegas, and perhaps the best...",1
7,bumAFxitMRHKAxZMijvUYg,"Caterers, Event Planning & Services, Restauran...",1,Apparently this used to be a restaurant before...,1
8,ohz8ljVxcZMo2bMGrBZfgw,"Vegetarian, Nightlife, Indian, Salad, Pakistan...",0,Nice little Indian restaurant near my hotel. H...,0
9,iJfS02AOptwkaWD-vOVRew,"Restaurants, Indian",0,"My husband & I LOVE this place! Service, food...",0


In [66]:
india_review=indian_rest_review_prediction[["business_id","text","label"]]

In [69]:
india_review[india_review["label"] == 1].count()

business_id    38041
text           38041
label          38041
dtype: int64

In [70]:
india_review[india_review["label"] == 0].count()

business_id    41726
text           41726
label          41726
dtype: int64

In [76]:

import nltk

#nltk.download("wordnet")
#nltk.download("stopwords")
import unicodedata
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
stopwords_ = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer() 
tf = CountVectorizer()

In [72]:
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode()


In [77]:
def pre_processing(review):
    india_review["normalizes_text"] = india_review["text"].apply(lambda x:remove_accents(x))
    #india_review["remove_html"] = india_review['normalizes_text'].apply(lambda x: strip_html_tags(x))
    india_review["remove_special"] = india_review["normalizes_text"].replace(r'[^A-Za-z0-9 ]+', '', regex=True)
    india_review["token_text"] = india_review["remove_special"].apply(lambda x:[word.lower() for word in x.split(" ")]) 
    india_review["remove_stop"] = india_review["token_text"].apply(lambda x:[word for word in x if word not  in stopwords_])
    india_review["lemitize_text"] = india_review["remove_stop"].apply(lambda x: " ".join([lemmatizer.lemmatize(word) for word in x]))
   # review["clean_list"] = review["lemitize_text"].apply(lambda x:[word for word in x.split(" ")])
    return india_review
     
  
    

In [ ]:
new_india_review=pre_processing(india_review)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
new_india_review.head(5)

In [ ]:
def word_counter(review):
    countvect = CountVectorizer()
    bag_of_words = countvect.fit_transform(review["lemitize_text"])
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in countvect.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq

In [ ]:
counter = word_counter(new_india_review)


In [ ]:
X = new_india_review["lemitize_text"]
X.head()
y=new_india_review["label"]
tfidf = TfidfVectorizer()
X_mat=tfidf.fit_transform(X)

In [ ]:
def predict_model(X,y,model):
    
    print("model is ",model)
    X_train,X_test,y_train,y_test = train_test_split(X_mat,y,test_size=0.20)
    #rf = RandomForestClassifier()
    
    
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    
    print("accuracy score = ",accuracy_score(y_test,y_pred))
    print("accuracy score = ",confusion_matrix(y_test,y_pred))
    print("precision = ",precision)
    print("recall = ",recall)
    print("f1 score = ",f1_score(y_test,y_pred))
    

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score,f1_score


rf = RandomForestClassifier()
GNB = GaussianNB()
MNB = MultinomialNB()
BNB = BernoulliNB()
lg = LogisticRegression()
dt = DecisionTreeClassifier()

# X = vectorizer.fit_transform(text_series)
#     X_train, X_test, y_train, y_test = train_test_split(X, y)
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_test)
#     precision = precision_score(y_test, y_pred)
#     recall = recall_score(y_test, y_pred)
#     accuracy = accuracy_score(y_test, y_pred)

models = [dt,rf,lg,MNB,BNB]

for model in models:
    predict_model(X,y,model)